# Migrate data from Oracle to Postgres

In [ ]:
# Import libraries
import sqlalchemy
import cx_Oracle
import getpass
import migrate

## 1. Connect to source database

In [ ]:
src_username = input('- Username on source database (default "sys"): ') or 'sys'
src_host = input('- Hostname for source database (default "localhost": ') or 'localhost'
src_port = input('- Port for source database (default "1521"): ') or 1521
src_database = input('- Name of source database (default "sys"): ') or 'sys'
src_password = getpass.getpass('- Password for source database: ')

dsn_str = cx_Oracle.makedsn(src_host,src_port,service_name=src_database)
src_con_string = 'oracle://{}:{}@'.format(src_username, src_password) + dsn_str
source_engine = sqlalchemy.create_engine(src_con_string)

## 2. Connect to target database

In [ ]:
# connect to target database
target_username = input('- Username on target database (default "postgres"): ') or 'postgres'
target_host = input('- Hostname for target database (default "localhost"): ') or 'localhost'
target_port = input('- Port for target database (default "5432"): ') or 5432
target_database = input('- Name of target database (default "postgres"): ') or 'postgres'
target_password = getpass.getpass('- Password for target database: ')
print_log = False

con_string_target = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(target_username, target_password, 
                                                                  target_host, target_port, target_database)
target_engine = sqlalchemy.create_engine(con_string_target, echo = print_log)

## 3. Create database on target system

In [ ]:
# create a new database on the target
# WARNING: deletes target database before creation!
msg =  """
        ----------------------------------------------------
        Running this script will delete the target database!\n
        Are you sure you wish to continue? (y/n)
        ----------------------------------------------------
        """
if input(msg) != "y":
    print('Not continuing')
else: 
    new_target_database = 'oracle_migration'
    migrate.drop_connections(new_target_database,target_engine)
    migrate.drop_database(new_target_database,target_engine)
    migrate.create_database(new_target_database,target_engine)

In [ ]:
# reconnect to this target database
con_string_target = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(target_username, target_password, 
                                                                  target_host, target_port, new_target_database)
target_engine = sqlalchemy.create_engine(con_string_target, 
                                         echo = print_log)

## 4. Get list of schema on source system

In [ ]:
# get list of all schema
inspector = sqlalchemy.inspect(source_engine)
schema_list = inspector.get_schema_names()
print(schema_list)

In [ ]:
# Omit the following schema from the migration
omit= ['']

## 5. Recreate source structure on target system

In [ ]:
# Iterate all of the schema on the database
for source_schema in schema_list:

    # skip schema in omit list
    if source_schema in omit:
        continue
    
    # load the schema metadata profile
    print(source_schema)
    source_metadata = sqlalchemy.MetaData(source_engine,quote_schema=True)
    source_metadata.reflect(schema=source_schema)

    # create the schema on the target database
    target_engine.execute(sqlalchemy.schema.CreateSchema(source_schema))

    # iterate the tables
    for t in source_metadata.sorted_tables:

        # clear the indexes and constraints
        t.indexes.clear()
        t.constraints.clear()
        
        # clean the data types
        for col in t.columns:
            
            # set the column types
            newtype = migrate.convert_type(col.name, col.type)
            t.c[col.name].type = newtype
            
            # check the default values
            if t.c[col.name].default:
                new_default = migrate.check_default(t.c[col.name].default)
                t.c[col.name].default = new_default

            # remove the server_default values
            if t.c[col.name].server_default:
                t.c[col.name].server_default = None            

    # Build the tables on the target database
    source_metadata.create_all(target_engine,checkfirst=False)

## 6. Load the data into the target system

In [ ]:
import pdb
# Iterate all of the schema on the database
# Migrate data in chunks
chunksize=100

for source_schema in schema_list:

    # skip schema in omit list
    if source_schema in omit:
        continue
    
    # load the schema metadata profile
    source_metadata = sqlalchemy.MetaData(source_engine)
    source_metadata.reflect(schema=source_schema)

    # iterate the tables, loading the data
    for t in source_metadata.sorted_tables:
        migrate.copy_data(source_engine,source_schema,target_engine,t,chunksize,debug=True)